## Įvadas

Šioje pamokoje aptarsime:
- Kas yra funkcijų kvietimas ir kur jis naudojamas
- Kaip sukurti funkcijų kvietimą naudojant Azure OpenAI
- Kaip integruoti funkcijų kvietimą į programą

## Mokymosi tikslai

Baigę šią pamoką, mokėsite ir suprasite:

- Kodėl verta naudoti funkcijų kvietimą
- Kaip nustatyti funkcijų kvietimą naudojant Azure Open AI paslaugą
- Kaip sukurti efektyvius funkcijų kvietimus pagal savo programos poreikius


## Suprasti funkcijų iškvietimus

Šioje pamokoje kursime funkciją mūsų švietimo startuoliui, kuri leis naudotojams per pokalbių robotą rasti techninius kursus. Rekomenduosime kursus, atitinkančius jų įgūdžių lygį, dabartinį vaidmenį ir dominančias technologijas.

Tam įgyvendinti naudosime šiuos komponentus:
 - `Azure Open AI`, kad sukurtume pokalbio patirtį naudotojui
 - `Microsoft Learn Catalog API`, kad padėtume naudotojams rasti kursus pagal jų užklausą
 - `Function Calling`, kad naudotojo užklausą perduotume funkcijai, kuri atliks API užklausą

Pradėkime nuo to, kodėl apskritai verta naudoti funkcijų iškvietimą:

print("Žinutės kitoje užklausoje:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # gauti naują GPT atsakymą, kuriame jis mato funkcijos atsakymą


print(second_response.choices[0].message)


### Kodėl verta naudoti funkcijų iškvietimą

Jei jau baigėte bent vieną kitą šio kurso pamoką, tikriausiai suprantate, kokią galią turi dideli kalbos modeliai (LLM). Tikimės, kad pastebėjote ir jų tam tikrus apribojimus.

Funkcijų iškvietimas – tai Azure Open AI Service funkcija, padedanti įveikti šiuos apribojimus:
1) Nuoseklus atsakymų formatas
2) Galimybė naudoti duomenis iš kitų programos šaltinių pokalbio kontekste

Prieš atsirandant funkcijų iškvietimui, LLM atsakymai būdavo neapibrėžti ir nenuoseklūs. Kūrėjams tekdavo rašyti sudėtingą validavimo kodą, kad galėtų apdoroti kiekvieną atsakymo variantą.

Vartotojai negalėdavo gauti atsakymų, tokių kaip „Koks dabar oras Stokholme?“. Taip yra todėl, kad modeliai buvo apriboti duomenų, kuriais buvo apmokyti, laikotarpiu.

Pažvelkime į žemiau pateiktą pavyzdį, kuris iliustruoja šią problemą:

Tarkime, norime sukurti studentų duomenų bazę, kad galėtume jiems pasiūlyti tinkamą kursą. Žemiau pateikti du studentų aprašymai, kurie yra labai panašūs pagal juose esančią informaciją.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finishing his studies."

Norime tai išsiųsti LLM, kad jis apdorotų duomenis. Vėliau tai galėsime panaudoti savo programėlėje, siųsti į API arba saugoti duomenų bazėje.

Sukurkime du identiškus užklausos tekstus, kuriais nurodome LLM, kokia informacija mums yra aktuali:


Mes norime tai nusiųsti LLM, kad išanalizuotų mūsų produktui svarbias dalis. Taigi galime sukurti du identiškus raginimus, kad nurodytume LLM:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


Sukūrę šiuos du raginimus, juos išsiųsime LLM naudodami `openai.ChatCompletion`. Raginimą saugome kintamajame `messages` ir priskiriame vaidmenį `user`. Tai daroma tam, kad būtų imituojama vartotojo žinutė, rašoma pokalbių robotui.


In [ ]:
import os
import json
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-07-01-preview"
  )

deployment=os.environ['AZURE_OPENAI_DEPLOYMENT']

: 

In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

Nors užklausos yra tokios pačios ir aprašymai panašūs, galime gauti skirtingus `Grades` savybės formatus.

Jei aukščiau esantį langelį paleisite kelis kartus, formatas gali būti `3.7` arba `3.7 GPA`.

Taip yra todėl, kad LLM priima nestruktūruotus duomenis rašytinės užklausos pavidalu ir taip pat grąžina nestruktūruotus duomenis. Mums reikia turėti struktūruotą formatą, kad žinotume, ko tikėtis saugant ar naudojant šiuos duomenis.

Naudodami funkcijų iškvietimą, galime užtikrinti, kad gausime atgal struktūruotus duomenis. Naudojant funkcijų iškvietimą, LLM iš tikrųjų neiškviečia ir nevykdo jokių funkcijų. Vietoj to, mes sukuriame struktūrą, kurios LLM turi laikytis savo atsakymuose. Tuomet tuos struktūruotus atsakymus naudojame tam, kad žinotume, kokią funkciją paleisti savo programose.


![Funkcijų iškvietimo srauto diagrama](../../../../translated_images/Function-Flow.083875364af4f4bb69bd6f6ed94096a836453183a71cf22388f50310ad6404de.lt.png)


### Funkcijų iškvietimo naudojimo atvejai

**Išorinių įrankių kvietimas**
Pokalbių robotai puikiai atsako į vartotojų klausimus. Naudojant funkcijų iškvietimą, pokalbių robotai gali panaudoti vartotojo žinutes tam tikroms užduotims atlikti. Pavyzdžiui, studentas gali paprašyti roboto: „Išsiųsk el. laišką mano dėstytojui, kad man reikia daugiau pagalbos su šia tema“. Tokiu atveju gali būti iškviečiama funkcija `send_email(to: string, body: string)`

**API ar duomenų bazės užklausų kūrimas**
Vartotojai gali ieškoti informacijos natūralia kalba, kuri paverčiama į suformatuotą užklausą ar API prašymą. Pavyzdžiui, mokytojas gali paklausti: „Kas iš studentų atliko paskutinę užduotį?“, ir tai gali iškviesti funkciją `get_completed(student_name: string, assignment: int, current_status: string)`

**Struktūruotų duomenų kūrimas**
Vartotojai gali paimti teksto bloką ar CSV ir pasitelkti LLM, kad išgautų svarbią informaciją. Pavyzdžiui, studentas gali konvertuoti Vikipedijos straipsnį apie taikos susitarimus ir sukurti AI atminties korteles. Tam galima naudoti funkciją `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)`


## 2. Pirmosios funkcijos iškvietimo kūrimas

Funkcijos iškvietimo kūrimo procesą sudaro 3 pagrindiniai žingsniai:
1. Iškvieskite Chat Completions API su savo funkcijų sąrašu ir vartotojo žinute
2. Perskaitykite modelio atsakymą, kad atliktumėte veiksmą, pvz., įvykdytumėte funkciją ar API užklausą
3. Dar kartą iškvieskite Chat Completions API su jūsų funkcijos atsakymu, kad galėtumėte panaudoti tą informaciją atsakymui vartotojui sukurti


![Funkcijos iškvietimo eiga](../../../../translated_images/LLM-Flow.3285ed8caf4796d7343c02927f52c9d32df59e790f6e440568e2e951f6ffa5fd.lt.png)


### Funkcijos iškvietimo elementai

#### Vartotojo įvestis

Pirmas žingsnis – sukurti vartotojo žinutę. Tai galima padaryti dinamiškai, paimant reikšmę iš teksto įvesties lauko, arba galite reikšmę priskirti čia. Jei pirmą kartą dirbate su Chat Completions API, turime apibrėžti žinutės `role` ir `content`.

`role` gali būti `system` (taisyklių kūrimas), `assistant` (modelis) arba `user` (galutinis vartotojas). Funkcijų iškvietimui priskirsime `user` ir pateiksime pavyzdinį klausimą.


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### Funkcijų kūrimas.

Toliau apibrėšime funkciją ir jos parametrus. Čia naudosime tik vieną funkciją, pavadintą `search_courses`, tačiau galite sukurti ir daugiau funkcijų.

**Svarbu**: Funkcijos yra įtraukiamos į sistemos žinutę LLM ir jos bus skaičiuojamos į jūsų turimų žetonų kiekį.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**Apibrėžimai**

`name` - Funkcijos, kurią norime iškviesti, pavadinimas.

`description` - Tai aprašymas, kaip veikia funkcija. Čia svarbu būti konkrečiam ir aiškiam.

`parameters` - Vertybių ir formato sąrašas, kurį norite, kad modelis pateiktų savo atsakyme.

`type` - Duomenų tipas, kuriame bus saugomos savybės.

`properties` - Konkretūs reikšmių sąrašai, kuriuos modelis naudos savo atsakyme.

`name` - Savybės pavadinimas, kurį modelis naudos suformatuotame atsakyme.

`type` - Šios savybės duomenų tipas.

`description` - Konkrečios savybės aprašymas.

**Nebūtina**

`required` - Būtina savybė, kad funkcijos iškvietimas būtų įvykdytas.


### Funkcijos iškvietimas
Apibrėžus funkciją, dabar turime ją įtraukti į Chat Completion API užklausą. Tai darome pridėdami `functions` prie užklausos. Šiuo atveju `functions=functions`.

Taip pat yra galimybė nustatyti `function_call` reikšmę į `auto`. Tai reiškia, kad leisime LLM pačiam nuspręsti, kuri funkcija turėtų būti iškviesta pagal vartotojo žinutę, užuot priskyrę ją patys.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

Dabar pažiūrėkime į atsakymą ir jo formatavimą:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

Matote, kad funkcijos pavadinimas yra iškviestas, o iš naudotojo žinutės LLM sugebėjo rasti duomenis, tinkamus funkcijos argumentams.


## 3. Funkcijų iškvietimų integravimas į programą.

Ištestavus suformatuotą LLM atsakymą, dabar galime jį integruoti į programą.

### Srauto valdymas

Norėdami tai integruoti į savo programą, atlikime šiuos veiksmus:

Pirmiausia, iškvieskime Open AI paslaugas ir išsaugokime žinutę kintamajame `response_message`.


In [ ]:
response_message = response.choices[0].message

Dabar apibrėšime funkciją, kuri iškvies Microsoft Learn API, kad gautų kursų sąrašą:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



Kaip geriausia praktika, mes pirmiausia patikrinsime, ar modelis nori iškviesti funkciją. Po to sukursime vieną iš galimų funkcijų ir pritaikysime ją prie kviečiamos funkcijos.

Tada paimsime funkcijos argumentus ir susiesime juos su argumentais iš LLM.

Galiausiai, pridėsime funkcijos iškvietimo žinutę ir reikšmes, kurias grąžino `search_courses` žinutė. Tai suteikia LLM visą reikiamą informaciją, kad galėtų atsakyti vartotojui natūralia kalba.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## Kodo iššūkis

Puikus darbas! Norėdami toliau gilinti žinias apie Azure Open AI funkcijų iškvietimą, galite sukurti: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst
 - Daugiau funkcijos parametrų, kurie galėtų padėti besimokantiesiems rasti daugiau kursų. Galimus API parametrus rasite čia:
 - Sukurkite dar vieną funkcijos iškvietimą, kuris gautų daugiau informacijos iš besimokančiojo, pavyzdžiui, jo gimtąją kalbą
 - Sukurkite klaidų valdymą, kai funkcijos ir/arba API iškvietimas negrąžina tinkamų kursų



---

**Atsakomybės atsisakymas**:  
Šis dokumentas buvo išverstas naudojant dirbtinio intelekto vertimo paslaugą [Co-op Translator](https://github.com/Azure/co-op-translator). Nors siekiame tikslumo, atkreipkite dėmesį, kad automatiniai vertimai gali turėti klaidų ar netikslumų. Originalus dokumentas jo gimtąja kalba turėtų būti laikomas autoritetingu šaltiniu. Svarbiai informacijai rekomenduojame profesionalų žmogaus vertimą. Mes neprisiimame atsakomybės už bet kokius nesusipratimus ar neteisingą interpretavimą, kilusį naudojantis šiuo vertimu.
